# Bangkok AirBnb Analysis

The goal for this project is to analyse data to understand what kinds of listings are likely to attract more occupants.

Data sourced from: https://insideairbnb.com

# Exploratory Data Analysis

In [70]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('/Users/joeytan/Documents/Data Analyst/Portfolio/PortfolioProject1_Airbnb/Datasets/Cleaned Data/listings_cleaned.csv')
df2 = pd.read_csv('/Users/joeytan/Documents/Data Analyst/Portfolio/PortfolioProject1_Airbnb/Datasets/Cleaned Data/calendar_cleaned.csv')

In [71]:
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17463 entries, 0 to 17462
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           17463 non-null  int64  
 1   host_response_time           16614 non-null  object 
 2   host_response_rate           16614 non-null  object 
 3   host_acceptance_rate         16466 non-null  object 
 4   host_is_superhost            16400 non-null  object 
 5   host_has_profile_pic         17463 non-null  object 
 6   host_identity_verified       17463 non-null  object 
 7   neighbourhood_cleansed       17463 non-null  object 
 8   latitude                     17463 non-null  float64
 9   longitude                    17463 non-null  float64
 10  property_type                17463 non-null  object 
 11  room_type                    17463 non-null  object 
 12  accommodates                 17463 non-null  int64  
 13  bathrooms       

# Seasonal Demand

## Average Price by month

In [74]:
df2['date'] = pd.to_datetime(df2['date'])
df2['month'] = df2['date'].dt.to_period('M')

avg_price_by_month = df2.groupby(['month'])['price'].mean().round().reset_index().rename(columns={'price': 'avg_price'})
avg_price_by_month

month  avg_price
0   2024-12     1577.0
1   2025-01     1578.0
2   2025-02     1578.0
3   2025-03     1578.0
4   2025-04     1578.0
5   2025-05     1578.0
6   2025-06     1578.0
7   2025-07     1578.0
8   2025-08     1578.0
9   2025-09     1578.0
10  2025-10     1578.0
11  2025-11     1578.0
12  2025-12     1578.0
13  2026-01     1714.0

## Average Occupancy rate by month

In [76]:
avg_occupancy_by_month = (
    df2.groupby('month')['available'] 
    .agg(lambda x: (x == 'f').mean() * 100) 
    .round(1)
    .reset_index()
    .rename(columns={'available': 'occupancy_rate(%)'})
)

avg_occupancy_by_month

month  occupancy_rate(%)
0   2024-12               80.7
1   2025-01               45.0
2   2025-02               25.7
3   2025-03               14.3
4   2025-04               18.8
5   2025-05               16.6
6   2025-06               19.4
7   2025-07               31.0
8   2025-08               30.7
9   2025-09               35.0
10  2025-10               46.7
11  2025-11               47.1
12  2025-12               47.3
13  2026-01               66.7

# Room Types

## Average price by room type

In [79]:
avg_prices_by_type = df1.groupby('room_type')['price'].mean().reset_index().sort_values(['price'],ascending=False)
    
avg_prices_by_type.rename(columns={'price': 'avg_price'})

room_type    avg_price
0  Entire home/apt  2095.273825
1       Hotel room  2011.989130
2     Private room  1570.780909
3      Shared room   583.469136

## Average Occupancy rate by room type

In [81]:
avg_occupancy_by_room_type = (
    df1.groupby('room_type', as_index=False)
    .agg(avg_occupancy_rate=('availability_365', 
                           lambda x: (365 - x).mean()/365 * 100))
    .round(1)
    .sort_values('avg_occupancy_rate', ascending=False)
)

avg_occupancy_by_room_type

room_type  avg_occupancy_rate
0  Entire home/apt                35.0
2     Private room                25.8
1       Hotel room                22.9
3      Shared room                16.7

## Average price by number of bedrooms

In [83]:
avg_prices_by_rooms =df1.groupby(['bedrooms'])['price'].mean().round().reset_index().sort_values(['price'],ascending=False)
avg_prices_by_rooms

bedrooms   price
5        5.0  3966.0
4        4.0  3472.0
3        3.0  3340.0
15      30.0  2949.0
2        2.0  2906.0
6        6.0  2567.0
9        9.0  1908.0
11      12.0  1771.0
1        1.0  1711.0
0        0.0  1398.0
10      10.0  1235.0
7        7.0  1033.0
16      50.0   800.0
13      16.0   711.0
12      15.0   677.0
14      20.0   645.0
8        8.0   598.0

Interesting... there are listings with 0 bedrooms, I'm guessing those are hostels with dormitories
## Average Occupancy rate by number of bedrooms

In [85]:
avg_occupancy_by_rooms = (
    df1.groupby(df1['bedrooms'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index().rename(columns={'availability_365':'occupancy rate'}))

avg_occupancy_by_rooms

bedrooms  occupancy rate
0        9.0            75.5
1       50.0            43.0
2        2.0            36.4
3        0.0            35.0
4        3.0            34.1
5        1.0            31.1
6        5.0            29.5
7        4.0            28.7
8       20.0            26.0
9       16.0            23.6
10       8.0            22.6
11       6.0            22.2
12      10.0            18.2
13      30.0            13.4
14      15.0            13.3
15       7.0            10.2
16      12.0             1.6

## Average price by capacity

In [87]:
avg_prices_by_cap =df1.groupby(['accommodates'])['price'].mean().round().reset_index().sort_values(['price'],ascending=False)
avg_prices_by_cap

accommodates   price
14            15  4814.0
12            13  4212.0
10            11  3799.0
6              7  3399.0
8              9  3351.0
9             10  3320.0
11            12  3262.0
7              8  3155.0
4              5  2938.0
5              6  2912.0
13            14  2878.0
3              4  2620.0
15            16  2615.0
2              3  1867.0
1              2  1652.0
0              1  1101.0

## Average Occupancy rate by capacity

In [89]:
avg_occupancy_by_cap = (
    df1.groupby(df1['accommodates'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'bedrooms': 'capacity','availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_cap

accommodates  avg_occupancy_rate(%)
0              7                   42.8
1             11                   37.9
2              5                   36.8
3              4                   36.2
4              3                   35.1
5             14                   33.2
6              6                   32.3
7              8                   31.4
8              2                   30.5
9             10                   30.0
10             9                   29.9
11             1                   28.3
12            12                   25.4
13            15                   22.5
14            13                   17.7
15            16                   15.8

# Neighbourhood

## Average price by neighbourhood

In [92]:
prices_by_neighbourhood = (df1.groupby(['neighbourhood_cleansed'])
                           ['price'].mean().round().reset_index()
                           .sort_values(['price'],ascending=False)
                          .rename(columns={'price':'avg_price'}))

prices_by_neighbourhood

neighbourhood_cleansed  avg_price
29            Parthum Wan     2640.0
8                Bang Rak     2425.0
47                Vadhana     2416.0
39         Samphanthawong     2291.0
18            Huai Khwang     2251.0
22            Khlong Toei     2222.0
37            Ratchathewi     2100.0
34   Pom Prap Sattru Phai     2053.0
33            Phra Nakhon     1970.0
41                 Sathon     1779.0
44         Thawi Watthana     1725.0
35                Pra Wet     1719.0
31             Phaya Thai     1717.0
20          Khlong Sam Wa     1679.0
27              Nong Chok     1679.0
25              Lat Phrao     1655.0
15              Din Daeng     1609.0
21             Khlong San     1595.0
49              Yan na wa     1578.0
5         Bang Khun thain     1564.0
32           Phra Khanong     1560.0
46             Thung khru     1548.0
11            Bangkok Yai     1528.0
13             Chatu Chak     1498.0
38                Sai Mai     1497.0
43            Taling Chan     1478.0
4           Bang Kho laen     1457.0
42              Suanluang     1418.0
19            Khan Na Yao     1415.0
14             Chom Thong     1352.0
24            Lat Krabang     1350.0
12              Bueng Kum     1333.0
45              Thon buri     1315.0
36             Rat Burana     1289.0
6                 Bang Na     1279.0
40            Saphan Sung     1279.0
3               Bang Khen     1250.0
17                  Dusit     1239.0
48        Wang Thong Lang     1234.0
26               Min Buri     1233.0
1               Bang Kapi     1226.0
16             Don Mueang     1202.0
30          Phasi Charoen     1163.0
9                Bang Sue     1160.0
0                Bang Bon     1149.0
10            Bangkok Noi     1132.0
23                 Lak Si     1061.0
2               Bang Khae     1041.0
7              Bang Phlat     1029.0
28             Nong Khaem     1007.0

## Average occupancy rate by neighbourhood

In [94]:
avg_occupancy_by_neighborhood = (
    df1.groupby(['neighbourhood_cleansed'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_neighborhood

neighbourhood_cleansed  avg_occupancy_rate(%)
0             Taling Chan                   45.8
1               Yan na wa                   40.3
2                 Vadhana                   38.4
3              Chom Thong                   37.0
4                  Sathon                   36.2
5             Ratchathewi                   35.5
6             Parthum Wan                   35.3
7                 Bang Na                   35.0
8             Khlong Toei                   34.6
9                Bang Sue                   34.4
10              Din Daeng                   34.1
11        Wang Thong Lang                   33.8
12             Khlong San                   33.7
13              Thon buri                   33.6
14           Phra Khanong                   33.3
15              Bueng Kum                   32.9
16   Pom Prap Sattru Phai                   32.2
17             Phaya Thai                   31.4
18               Bang Rak                   30.9
19              Nong Chok                   30.8
20             Rat Burana                   30.6
21            Bangkok Noi                   29.8
22            Khan Na Yao                   29.0
23          Phasi Charoen                   28.6
24             Chatu Chak                   28.6
25            Huai Khwang                   28.5
26            Lat Krabang                   27.8
27              Bang Kapi                   27.5
28             Don Mueang                   27.4
29              Bang Khen                   27.2
30             Bang Phlat                   27.1
31               Min Buri                   26.7
32                Pra Wet                   26.3
33         Samphanthawong                   25.9
34          Bang Kho laen                   25.7
35                 Lak Si                   24.2
36             Thung khru                   23.2
37        Bang Khun thain                   23.0
38            Bangkok Yai                   22.9
39              Suanluang                   22.3
40              Lat Phrao                   18.1
41            Phra Nakhon                   18.0
42         Thawi Watthana                   17.5
43              Bang Khae                   17.2
44                  Dusit                   17.2
45                Sai Mai                   14.9
46            Saphan Sung                   14.0
47             Nong Khaem                   10.9
48          Khlong Sam Wa                    9.2
49               Bang Bon                    7.2

## Count of listings in each neighbourhood

In [96]:
neighbourhood_density = (df1.groupby(['neighbourhood_cleansed'])
                         ['id'].nunique().reset_index()
                         .sort_values(by='id',ascending = False)
                        .rename(columns={'id':'listings_count'}))

neighbourhood_density

neighbourhood_cleansed  listings_count
47                Vadhana            2800
22            Khlong Toei            2468
18            Huai Khwang            2065
37            Ratchathewi            1046
41                 Sathon             832
32           Phra Khanong             802
33            Phra Nakhon             660
8                Bang Rak             583
13             Chatu Chak             543
42              Suanluang             513
15              Din Daeng             405
6                 Bang Na             399
21             Khlong San             388
29            Parthum Wan             387
31             Phaya Thai             306
9                Bang Sue             306
7              Bang Phlat             285
1               Bang Kapi             283
45              Thon buri             217
49              Yan na wa             174
24            Lat Krabang             166
10            Bangkok Noi             140
16             Don Mueang             135
34   Pom Prap Sattru Phai             117
48        Wang Thong Lang             114
30          Phasi Charoen             109
23                 Lak Si             108
4           Bang Kho laen             106
39         Samphanthawong             103
3               Bang Khen             100
35                Pra Wet              92
11            Bangkok Yai              80
17                  Dusit              68
40            Saphan Sung              64
12              Bueng Kum              64
2               Bang Khae              56
14             Chom Thong              53
36             Rat Burana              50
19            Khan Na Yao              47
25              Lat Phrao              43
26               Min Buri              39
43            Taling Chan              35
38                Sai Mai              32
5         Bang Khun thain              17
44         Thawi Watthana              15
20          Khlong Sam Wa              15
46             Thung khru              11
27              Nong Chok               8
28             Nong Khaem               7
0                Bang Bon               7

## Average review scores by neighbourhood

In [98]:
neighbourhood_by_review_scores=(df1.groupby(['neighbourhood_cleansed'])['review_scores_location']
                                .mean().round(2).reset_index().sort_values(['review_scores_location'], ascending=False))

neighbourhood_by_review_scores

neighbourhood_cleansed  review_scores_location
8                Bang Rak                    4.80
29            Parthum Wan                    4.79
43            Taling Chan                    4.79
33            Phra Nakhon                    4.77
39         Samphanthawong                    4.75
11            Bangkok Yai                    4.75
47                Vadhana                    4.74
21             Khlong San                    4.73
46             Thung khru                    4.73
34   Pom Prap Sattru Phai                    4.71
22            Khlong Toei                    4.71
37            Ratchathewi                    4.71
17                  Dusit                    4.68
45              Thon buri                    4.67
31             Phaya Thai                    4.66
32           Phra Khanong                    4.66
23                 Lak Si                    4.66
15              Din Daeng                    4.66
13             Chatu Chak                    4.65
35                Pra Wet                    4.64
3               Bang Khen                    4.63
18            Huai Khwang                    4.62
24            Lat Krabang                    4.62
16             Don Mueang                    4.60
41                 Sathon                    4.60
12              Bueng Kum                    4.60
9                Bang Sue                    4.60
6                 Bang Na                    4.60
25              Lat Phrao                    4.58
1               Bang Kapi                    4.57
4           Bang Kho laen                    4.55
19            Khan Na Yao                    4.54
14             Chom Thong                    4.54
2               Bang Khae                    4.53
30          Phasi Charoen                    4.52
0                Bang Bon                    4.50
42              Suanluang                    4.49
7              Bang Phlat                    4.48
5         Bang Khun thain                    4.48
10            Bangkok Noi                    4.47
20          Khlong Sam Wa                    4.46
36             Rat Burana                    4.44
40            Saphan Sung                    4.44
48        Wang Thong Lang                    4.43
49              Yan na wa                    4.43
26               Min Buri                    4.42
38                Sai Mai                    4.39
44         Thawi Watthana                    4.35
27              Nong Chok                    4.00
28             Nong Khaem                     NaN

# Reviews

## Average price by review scores

In [124]:
avg_price_by_scores = (df1.groupby(round(df1['review_scores_rating']))
                           ['price'].mean().round().reset_index()
                           .sort_values(['price'],ascending=False)
                          .rename(columns={'price':'avg_price'}))

avg_price_by_scores

review_scores_rating  avg_price
4                   5.0     1923.0
1                   2.0     1893.0
3                   4.0     1870.0
0                   1.0     1768.0
2                   3.0     1707.0

## Average Occupancy rate by review scores

In [54]:
avg_occupancy_by_scores = (
    df1.groupby(round(df1['review_scores_rating']))['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_scores

review_scores_rating  avg_occupancy_rate(%)
0                   5.0                   37.0
1                   4.0                   31.8
2                   2.0                   29.2
3                   1.0                   27.2
4                   3.0                   26.6

## Average Occupancy rate by number of reviews

In [57]:
avg_occupancy_by_no_of_reviews = (
    df1.groupby(round(df1['number_of_reviews']/100)*100)['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'number_of_reviews': 'number_of_reviews(100s)','availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_no_of_reviews

number_of_reviews(100s)  avg_occupancy_rate(%)
0                     100.0                   40.9
1                     200.0                   39.1
2                     300.0                   36.8
3                     400.0                   32.4
4                       0.0                   30.9
5                     500.0                   24.0
6                     600.0                   22.6
7                     700.0                    9.3
8                    1400.0                    9.0
9                    1000.0                    8.1
10                   2400.0                    7.1
11                   1300.0                    4.1
12                    800.0                    3.8
13                   1100.0                    3.0

# Superhosts

## Superhost vs Regular host average price

In [169]:
avg_price_by_super = (
    df1.groupby(df1['host_is_superhost'])['price'].mean().sort_values(ascending=False).reset_index().rename(columns={'price': 'avg_price'})
)
avg_price_by_super

host_is_superhost    avg_price
0                 f  1938.765003
1                 t  1904.385447

## Superhost vs Regular host average occupancy rate

In [166]:
avg_occupancy_by_super = (
    df1.groupby(df1['host_is_superhost'])['availability_365']
    .apply(lambda x: (365 - x).mean()/365 * 100)
    .sort_values(ascending=False)
    .round(1)
    .reset_index()
    .rename(columns={'availability_365': 'avg_occupancy_rate(%)'})
)
avg_occupancy_by_super

host_is_superhost  avg_occupancy_rate(%)
0                 t                   37.1
1                 f                   28.6

## Superhost factors

### Response time

In [128]:
superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby('host_response_time')
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby('host_response_time')
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

Superhost (%)  non_superhost(%)
host_response_time                                 
within an hour               88.3              70.9
within a few hours           10.4              13.2
within a day                  1.3               5.7
a few days or more            0.0              10.2

### Response rate

In [129]:
df1['host_response_rate'] = df1['host_response_rate'].replace('%', '', regex=True).astype(float)

superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['host_response_rate']/10)*10)
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['host_response_rate']/10)*10)
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'],ascending = False)

Superhost (%)  non_superhost(%)
host_response_rate                                 
100.0                        95.2              74.6
90.0                          3.6               6.5
80.0                          0.6               4.6
60.0                          0.4               1.2
70.0                          0.2               1.7
50.0                          0.1               1.5
0.0                           0.0               4.9
20.0                          0.0               0.9
10.0                          NaN               2.7
30.0                          NaN               0.6
40.0                          NaN               0.7

### Review scores

In [130]:
superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['review_scores_rating']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['review_scores_rating']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

Superhost (%)  non_superhost(%)
review_scores_rating                                 
5.0                            95.1              73.7
4.0                             4.5              22.2
3.0                             0.2               2.2
1.0                             0.1               1.1
2.0                             0.1               0.8

### Host verification

In [131]:
superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['host_identity_verified']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['host_identity_verified']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

Superhost (%)  non_superhost(%)
host_identity_verified                                 
t                                99.1              98.8
f                                 0.9               1.2

### Host has profile picture

In [132]:
superhost = (
    df1[df1['host_is_superhost'] == 't']
    .groupby(round(df1['host_has_profile_pic']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('Superhost (%)')
)

non_superhost = (
    df1[df1['host_is_superhost'] == 'f']
    .groupby(round(df1['host_has_profile_pic']))
    .size()
    .transform(lambda x: 100 * x / x.sum())
    .round(1)
    .rename('non_superhost(%)')
)

pd.concat([superhost, non_superhost], axis=1).sort_values(['Superhost (%)'], ascending = False)

Superhost (%)  non_superhost(%)
host_has_profile_pic                                 
t                              99.2              96.5
f                               0.8               3.5

# Conclusion

## Seasonal Demand
Occupancy rate is highest in Q4: October, November & December.

## Room types
Entire homes command the highest prices, and are the most likely to be booked.\
Listings with 3-5 bedrooms command the highest prices.\
Listings that accomodates 7 people are most likely to be booked.

## Neighbourhood
Listings located in Parthum Wan, Bang Rak and Vadhana command the highest prices, and they all have an average location rating above 4.7 indicating customers' satisfaction of location.\
Vadhana is the densest neighbourhood with 2,800 listings.

## Reviews
Listings with 5 stars command the highest prices and are the most likely to be booked.\
Number of reviews do not seem to influence occupancy rate.

## Superhost
Superhost status does not seem to influence average price of listings.\
However, superhosts' listings are 9% more likely to be booked.\
To be a superhost, one should have a high response rate and try to respond within 1 hour, he should also maintain a high review score of 5 stars.